In [75]:
from collections import namedtuple
import re
from typing import Dict, List, Optional

import clickhouse_connect
import fuzzywuzzy as fz
from geonorm.geonormaliser_utils import decompose
import pandas as pd
import tqdm

In [226]:
Owner = namedtuple("Owner", ["name", "country_code", "individual"])
Patent = namedtuple("Patent", ["number", "owners", "address"])
Person = namedtuple("Person", ["name", "tax_number"])

In [227]:
client = clickhouse_connect.get_client(
    host="localhost", 
    username="phd",
    password="phd",
)
client.command("SELECT version()")

'24.5.1.1763'

In [228]:
def parse(row: pd.Series) -> Patent:
    row = row.fillna("")
    owner_names = list(map(str.strip, row["patent holders"].split("\n")))
    address = row["correspondence address"]
    number = row["registration number"]
    
    regex = re.compile("\((?a:\w{2})\)")
    owners = []
    for name in owner_names:
        country_code = None
        individual = False
        
        country_code_match = regex.search(name)
        if country_code_match is not None:
            country_code = country_code_match.group(0)[1:-1]
            name = name.replace(country_code_match.group(0), "").strip()
        
        if row["authors"] == row["patent holders"]:
            individual = True
        
        name_parts = list(filter(lambda x: len(x) > 0, map(str.strip, name.split(" "))))
        if (
            len(name_parts) == 3
            and all(part[0].isupper() for part in name_parts)
        ):
            individual = True
        if (
            len(name_parts) == 2 
            and name_parts[0][0].isupper()
            and name_parts[1].replace(".", "").isupper()
        ):
            individual = True
        
        owners.append(Owner(name, country_code, individual))
        
    return Patent(number=number, owners=owners, address=address)

In [229]:
def search_by_exact_name_match(name: str, address: str) -> List[Person]:
    if name is None or address is None:
        return []
    
    stmt = """
        SELECT
            name,
            tax_number,
            ngramDistance(legal_address, {address:String}) as dal,
            ngramDistance(fact_address, {address:String}) as daf,
            dal * daf as score
        FROM search.search_base 
        WHERE name = {name:String}
        ORDER BY score
        LIMIT 2
    """
    
    params = {
        "name": name.upper(), 
        "address": address
    }    
    
    res = client.query(stmt, parameters=params)

    if len(res.result_rows) == 0:
        return []
    else:
        return [
            Person(name=row[0], tax_number=row[1])
            for row in res.result_rows
        ]

In [230]:
search_by_exact_name_match("МЕЛЬНИЧЕНКО ИГОРЬ ЮРЬЕВИЧ", "121165 Москва а/я 15 ООО \"Юстис\" Грунина А.Е.")

[Person(name='МЕЛЬНИЧЕНКО ИГОРЬ ЮРЬЕВИЧ', tax_number='771670514800')]

In [231]:
def unquote_name(name: str) -> str:
    name = name.replace("«", '"')
    name = name.replace("»", '"')
    
    if '"' not in name:
        return name
    
    parts = name.split('"')
    if len(parts) <= 4:
        return parts[1]
    else:
        return parts[2]

def search_by_like_name_match(name: str, address: Dict[str, str]) -> List[Person]:  
    stmt = """
        SELECT
            name,
            tax_number,
            ngramDistance(name, {original_name:String}) as dn,
            ngramDistance(legal_address, {address:String}) as dal,
            ngramDistance(fact_address, {address:String}) as daf,
            dn * dal * daf as score
        FROM search.search_base 
        WHERE name LIKE {unquoted_name:String}
        ORDER BY score
        LIMIT 1
    """
    
    params = {
        "original_name": name.upper(),
        "unquoted_name": f"%{unquote_name(name).upper()}%", 
        "address": address
    }     
    
    res = client.query(stmt, parameters=params)

    if len(res.result_rows) == 0:
        return []
    else:
        return [
            Person(name=row[0], tax_number=row[1])
            for row in res.result_rows
        ]

In [232]:
(
    unquote_name('Закрытое акционерное общество "Кыштымский медеэлектролитный завод"'),
    unquote_name("Государственное унитарное предприятие Издательство\"Советская Кубань\"")
)

('Кыштымский медеэлектролитный завод', 'Советская Кубань')

In [233]:
search_by_like_name_match(
    'Закрытое акционерное общество "Кыштымский медеэлектролитный завод"',
    '456870, Челябинская обл., г. Кыштым, ул. П.Коммуны, 2 Плеханову И.Д'
)

[Person(name='АКЦИОНЕРНОЕ ОБЩЕСТВО "КЫШТЫМСКИЙ МЕДЕЭЛЕКТРОЛИТНЫЙ ЗАВОД"', tax_number='7413000630')]

In [234]:
search_by_like_name_match("Нижегородский научно-исследовательский институт радиотехники", "")

[Person(name='ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ "НИЖЕГОРОДСКИЙ НАУЧНО-ИССЛЕДОВАТЕЛЬСКИЙ ИНСТИТУТ РАДИОТЕХНИКИ"', tax_number='5261000043')]

In [235]:
def search_by_tokens_match(name: str, address: Dict[str, str]) -> Optional[Person]:
    return None

In [236]:
def search(patent: Patent) -> List[Person]:
    result = []
    
    for owner in patent.owners:
        not_found = Person(owner.name, None)
        if owner.country_code and owner.country_code != "RU":
            result.append(not_found)
            continue
        if owner.individual is True:
            methods = (
                search_by_exact_name_match,
            )
        else:
            methods = (
                search_by_exact_name_match,
                search_by_like_name_match,
            )
        
        for method in methods:            
            found = method(owner.name, patent.address)
            
            if len(found) == 0:
                continue
            else:
                result.append(found[0])
                break
        else:
            result.append(not_found)
    
    return result

In [166]:
inv_sample = pd.read_csv("../data/opendata/samples/inventions_sample.csv")
mod_sample = pd.read_csv("../data/opendata/samples/models_sample.csv")
des_sample = pd.read_csv("../data/opendata/samples/designs_sample.csv")

In [244]:
def test_sample(df):
    result = []
    for _, row in tqdm.tqdm(df.iterrows()):
        patent = parse(row)
        persons = search(patent)
        for owner, person in zip(patent.owners, persons):
            result.append(
                (
                    patent.number, patent.address, person.name, person.tax_number,
                    owner.name, owner.individual, owner.country_code
                )
            )
    
    return pd.DataFrame(
        result, 
        columns=[
            "patent_number", "cor_address", "name", "tax_number", 
            "name_from_patent", "individual", "country"
        ]
    )

In [245]:
inv_test_result = test_sample(inv_sample)
inv_test_result.head()       

1000it [13:35,  1.23it/s]


,patent_number,cor_address,name,tax_number,name_from_patent,individual,country
0,2137261,"127560, Москва, ул.Коненкова, 5, кв.16, Демидо...",ДЕМИДОВ ЮРИЙ ПЕТРОВИЧ,503409279199,Демидов Юрий,False,None
1,2137261,"127560, Москва, ул.Коненкова, 5, кв.16, Демидо...",XL,7710140407,,False,None
2,2137261,"127560, Москва, ул.Коненкова, 5, кв.16, Демидо...",МИХАЙЛОВИЧ ВЛАДАН,771385293624,Михайлович,False,None
3,2631279,"141191, Московская обл., г. Фрязино, ул. Горьк...",Кочетов Олег Савельевич,None,Кочетов Олег Савельевич,True,RU
4,2731963,"119991, Москва, ГСП-1, ул. Ломоносовский просп...",ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,None,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,False,RU


In [247]:
inv_test_result["has_tn"] = inv_test_result["tax_number"].notna()
inv_test_result.groupby(["individual", "has_tn"]).size()

individual  has_tn
False       False     331
            True      449
True        False     391
            True      174
dtype: int64

In [250]:
inv_test_result.loc[
    (~inv_test_result["has_tn"] & ~inv_test_result["individual"]),
    ["name_from_patent", "cor_address"]
]["name_from_patent"].to_list()

['ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "НАЦИОНАЛЬНЫЙ МЕДИЦИНСКИЙ ИССЛЕДОВАТЕЛЬСКИЙ ЦЕНТР ЭНДОКРИНОЛОГИИ" МИНИСТЕРСТВА ЗДРАВООХРАНЕНИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ (ФГБУ "НМИЦ ЭНДОКРИНОЛОГИИ" МИНЗДРАВА РОССИИ)',
 'Федеральное государственное бюджетное образовательное учреждение высшего профессионального образования "Национальный минерально-сырьевой университет "Горный"',
 'Сибирский физико-технический институт при Томском государственном университете',
 'Российская Федерация, от имени которой выступает Министерство обороны Российской Федерации (Минобороны России)',
 'Акционерное общество "Астрата"',
 'ДАЛЯНЬ СЕЙФ ТЕХНОЛОДЖИ КО., ЛТД',
 'Производственное республиканское унитарное предприятие "Завод полупроводниковых приборов"',
 'Федеральное государственное бюджетное учреждение "Национальный медицинский исследовательский центр реабилитации и курортологии" Министерства здравоохранения Российской Федерации (ФГБУ "НМИЦ РК" Минздрава России)',
 'Пермский государственный университет',
 'О

In [252]:
inv_test_result.to_excel("inv_test_result.xlsx")

In [253]:
mod_test_result = test_sample(mod_sample)
mod_test_result.head()

1000it [04:40,  3.57it/s]


,patent_number,cor_address,name,tax_number,name_from_patent,individual,country
0,120365,"660133, г.Красноярск, ул. Авиаторов, 1, стр.1,...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АРНИКА""",2460018787,"Общество с ограниченной ответственностью ""Арника""",False,RU
1,32373,"198328, Санкт-Петербург, пр-т маршала Захарова...",БЕЛЯЕВ АЛЕКСАНДР ГЕННАДЬЕВИЧ,780708385189,Беляев Александр Геннадьевич,True,RU
2,96928,"117405, Москва, Варшавское ш., 143, корп.1, кв...","АКЦИОНЕРНОЕ ОБЩЕСТВО ""СВЯЗЬ ИНЖИНИРИНГ М""",7713551934,"Закрытое акционерное общество ""Связь инжинирин...",False,RU
3,116992,"141103, Московская обл., г. Щелково-3, ул. Гаг...",Филиппов Валерьян Степанович,None,Филиппов Валерьян Степанович,True,RU
4,169346,"680031, г. Хабаровск, ул. Карла Маркса, 144а, ...",БЕЗМАТЕРНЫХ РОМАН ВИКТОРОВИЧ,272511823375,Безматерных Роман Викторович,True,RU


In [254]:
mod_test_result["has_tn"] = mod_test_result["tax_number"].notna()
mod_test_result.groupby(["individual", "has_tn"]).size()

individual  has_tn
False       False     155
            True      563
True        False     225
            True      283
dtype: int64

In [259]:
mod_test_result.to_excel("mod_test_result.xlsx")

In [256]:
des_test_result = test_sample(des_sample)
des_test_result.head()

1000it [03:19,  5.02it/s]


,patent_number,cor_address,name,tax_number,name_from_patent,individual,country
0,81851,"191186, Санкт-Петербург, а/я 230, АРС-ПАТЕНТ, ...",Геберит Интернешенл АГ,None,Геберит Интернешенл АГ,True,CH
1,110832,"142455, \nМосковская обл., \nНогинский р-н, \n...","П.П.Х. ""АДАМЕКС"" Я.Каронь, Э.Каспшык, А. Каспш...",None,"П.П.Х. ""АДАМЕКС"" Я.Каронь, Э.Каспшык, А. Каспш...",False,PL
2,139296,"124460, \nМосква, \nг. Зеленоград, \nа/я 200,\...","АКЦИОНЕРНОЕ ОБЩЕСТВО ""КАМА""",1650404549,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""КАМА""",False,RU
3,94890,"660111, г.Красноярск, ул. Пограничников, 37, с...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ОБЪЕ...",3804039638,"Общество с ограниченной ответственностью ""Объе...",False,RU
4,125266,"117042, \nМосква, \nПлавский проезд, д. 1, кв....",Индивидуальный предприниматель Бирюков Денис В...,None,Индивидуальный предприниматель Бирюков Денис В...,False,RU


In [257]:
des_test_result["has_tn"] = des_test_result["tax_number"].notna()
des_test_result.groupby(["individual", "has_tn"]).size()

individual  has_tn
False       False     300
            True      343
True        False     276
            True      148
dtype: int64

In [260]:
des_test_result.to_excel("des_test_result.xlsx")